# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4.1'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nanonets/Nanonets-OCR-s',
 '/MiniMaxAI/MiniMax-M1-80k',
 '/google/magenta-realtime',
 '/mistralai/Mistral-Small-3.2-24B-Instruct-2506',
 '/Menlo/Jan-nano',
 '/models',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/enzostvs/deepsite',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces/MiniMaxAI/MiniMax-M1',
 '/spaces/nvidia/PartPacker',
 '/spaces',
 '/datasets/EssentialAI/essential-web-v1.0',
 '/datasets/institutional/institutional-books-1.0',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/nvidia/AceReason-1.1-SFT',
 '/datasets/nvidia/Nemotron-Personas',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers'

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community/discussion page',
   'url': 'https://discuss.huggingface.co'},
  {'type': 'social media - LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'brand/press page', 'url': 'https://huggingface.co/brand'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
nanonets/Nanonets-OCR-s
Updated
4 days ago
•
177k
•
1.1k
MiniMaxAI/MiniMax-M1-80k
Update

In [14]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [24]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [25]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'brand assets page', 'url': 'https://huggingface.co/brand'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'social page - LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnanonets/Nanonets-OCR-s\nUpdated\n4 days ago\n•\n177k\n•\n1.1k\nMiniMaxAI/MiniMax-M1-80k\nUpdated\nabout 5 hours ago\n•\n10.4k\n•\n544\ngoogle/magenta-realtime\nUpdated\n1 day ago\n•\n256\nmistralai/Mistral-Small-3.2-24B-Instruct-2506\nUpdated\n2 days ago\n•\n5.37k\n•\n207\nMenlo/Jan-nano\nUpdated\n7 days ago\n•\n29.1k\n•\n386\nBrowse 1M+ models\nSpaces\nRunning\n750\n750\nSparc3D\n🏃\nNext

In [26]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    # Clean up the response to remove code block formatting
    result = result.replace("```", "").replace("markdown", "")
    display(Markdown(result))

In [27]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'brand/media page', 'url': 'https://huggingface.co/brand'}, {'type': 'community/discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Welcome to Hugging Face

## The AI Community Building the Future

Hugging Face is the leading open platform where the global machine learning (ML) community collaborates, innovates, and builds the next generation of artificial intelligence. Whether you are a researcher, enterprise, or enthusiast, Hugging Face offers everything you need to create, discover, and deploy state-of-the-art ML models, datasets, and applications.

---

## What We Offer

### 🚀 Vast AI Ecosystem
- **1M+ Models:** Discover and use pre-trained models spanning text, vision, audio, and more.
- **250K+ Datasets:** Access a rich variety of datasets for every ML task.
- **400K+ Applications ("Spaces"):** Explore interactive, real-world AI apps built by the community.

### 💡 Open Source Foundations
Hugging Face is at the heart of open-source ML tooling:
- **Transformers:** The most popular library for PyTorch, TensorFlow, and JAX, powering natural language processing and beyond.
- **Diffusers, Tokenizers, PEFT, TRL:** Cutting-edge libraries for diffusion models, efficient tokenization, parameter-efficient finetuning, and reinforcement learning.
- **Safetensors, Hub Python Library, Transformers.js:** Tools for safe, fast, and portable ML workflows.

### 🏢 Enterprise Solutions
Scale and secure your AI efforts with Hugging Face for Teams and Enterprises:
- **Enterprise-Grade Security:** Single Sign-On (SSO), custom region & data management, audit logs, resource groups, and token management.
- **Dedicated Support:** Flexible billing, responsive support, and powerful infrastructure.

Starting at $20/user/month.

---

## Who Uses Hugging Face?

More than 50,000 organizations across the world rely on Hugging Face:
- **Tech Giants:** Google, Microsoft, Amazon, Meta, Intel
- **AI Leaders:** Ai2, Grammarly, Writer, and more
- **Startups & Non-Profits:** Everyone innovating in AI and ML

---

## Community & Company Culture

### 🌟 A Place to Collaborate and Grow
- **Open, Inclusive, and Developer-Driven:** Hugging Face thrives on community contributions and shared learning.
- **Build Your Portfolio:** Publish your work, showcase models & applications, and connect globally.

### 💬 Always-Evolving Forum
- **Active Discussions:** Collaborate with diverse experts via forums, Discord, and social platforms.
- **Learning Resources:** Extensive docs, guides, and a vibrant blog to support all skill levels.

---

## Careers at Hugging Face

Join a mission-driven team committed to democratizing AI and open source. Positions span machine learning, engineering, community management, product design, and more.

> _“We're building the home of machine learning. Your ideas, energy, and passion can help shape the AI future.”_

#### Explore Careers: [huggingface.co/jobs](https://huggingface.co/jobs)

---

## Get Started With Us

- **Browse Models:** [1M+ ready-to-use models](https://huggingface.co/models)
- **Try AI Apps:** [Explore Spaces](https://huggingface.co/spaces)
- **Enterprise Inquiries:** [Contact Sales](https://huggingface.co/contact)
- **Join the Community:** Sign up and be part of the world's fastest-growing AI hub.

> **Hugging Face — The Future of AI, Built Together**

[Website](https://huggingface.co/) | [GitHub](https://github.com/huggingface) | [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://www.linkedin.com/company/huggingface) | [Discord](https://discord.gg/huggingface)



## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [28]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [29]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'brand/resources page', 'url': 'https://huggingface.co/brand'}, {'type': 'documentation/resources', 'url': 'https://huggingface.co/docs'}, {'type': 'community/discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'social/linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face  
## The AI Community Building the Future

---

### Who We Are

Hugging Face is the epicenter of the global machine learning (ML) community, empowering millions of users to create, share, and collaborate on cutting-edge AI models, datasets, and applications. As the world’s leading open-source platform for AI, we connect researchers, developers, organizations, and enthusiasts to accelerate innovation in machine learning for everyone.

---

### What We Offer

- **AI Models & Datasets**  
  Access and contribute to a library of over **1 million AI models** and **250,000 datasets** covering all modalities: text, images, video, audio, and 3D.

- **Spaces**  
  Discover or showcase thousands of interactive AI applications—run by community members as well as leading tech companies—demonstrating the real-world impact of ML.

- **Collaboration Platform**  
  Host and collaborate on unlimited public models, datasets, and applications. Build your portfolio, receive feedback, and gain visibility in the global ML community.

- **Open Source Excellence**  
  We are committed to open-source, maintaining foundational tools like:
    - **Transformers** (PyTorch, TensorFlow, JAX)
    - **Diffusers** (State-of-the-art diffusion models)
    - **Safetensors** (Safe model storage)
    - **Tokenizers**, **Datasets**, and many more.

- **Enterprise Solutions**  
  Scale your AI capabilities with advanced, secure, and customizable solutions including:
    - **Single Sign-On (SSO)**
    - Region & data management
    - Audit logs & granular access control
    - Dedicated support and private dataset viewers

- **ML Compute & Inference**  
  Deploy and scale AI with optimized GPU compute and high-performance inference endpoints, starting at affordable pricing for organizations of all sizes.

---

### Who Trusts Us

Over **50,000 organizations** use Hugging Face, including leaders like:
- **Meta**
- **Google**
- **Amazon**
- **Intel**
- **Microsoft**
- **Grammarly**  
and many academic and non-profit institutions.

---

### Our Culture & Community

At Hugging Face, community is at the heart of everything:
- **Open Collaboration:** We believe in transparent, community-driven innovation. Contributors from all backgrounds help shape the future of AI.
- **Diversity in AI:** Our community includes researchers, data scientists, app developers, educators, and creative technologists worldwide.
- **Continuous Growth:** We support learning and experimentation, welcoming individuals wanting to build skills—or build the next breakthrough AI model.
- **Join the Conversation:** Engage with us on [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/huggingface), and [Discord](https://discord.gg/huggingface).

---

### Careers at Hugging Face

Help us democratize machine learning!  
We’re always searching for passionate, creative minds in engineering, research, developer advocacy, and beyond.  
- **Why Join?**
  - Work on impactful, open-source projects with global visibility
  - Collaborate with leaders in AI and MLOps
  - Flexible remote opportunities and a supportive, mission-driven team

Explore open positions on our [Jobs](https://huggingface.co/jobs) page.

---

### Ready to Explore?

- **[Sign Up](https://huggingface.co/join)** and start building your ML portfolio.
- **[Browse Models](https://huggingface.co/models)**, datasets, and applications.
- **[Contact Sales](https://huggingface.co/enterprise)** for enterprise-grade AI solutions.

---

> Hugging Face: Powering the global AI community—today and into the future.



In [30]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community/discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'social page - LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'social page - Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'social page - GitHub', 'url': 'https://github.com/huggingface'}]}



# Hugging Face: The AI Community Building the Future

---

## Who We Are

**Hugging Face** is the home of machine learning—an open collaborative platform where over one million models, 250,000 datasets, and 400,000+ AI applications are shared, built, and advanced by the global AI community. We are passionate about open source and making state-of-the-art ML accessible, safe, and impactful for everyone.

---

## What We Offer

### Platform & Community

- **Models**: Explore, host, and deploy more than 1,000,000 machine learning models, including text, image, audio, video, and 3D.
- **Datasets**: Access over 250,000 public datasets spanning every ML use case.
- **Spaces**: Discover 400,000+ interactive AI applications, or build and share your own.
- **Collaboration**: Create, share, and collaborate on projects, building your ML portfolio. Engage with the most active and diverse AI community in the world.

### Tools & Open Source Projects

- **Transformers**: Industry-leading library for working with large language models in PyTorch, TensorFlow, and JAX.
- **Diffusers**: Simplify and speed up working with diffusion models, including generative art and media.
- **Datasets Library**: Access and contribute to high-quality datasets in ML.
- **Tokenizers, TRL, PEFT, and more**: Tools designed for speed, efficiency, and research-friendly workflows.

### Enterprise Solutions

Empower your organization to scale AI securely:

- **Team & Enterprise Hubs**: Advanced access control, audit logs, SSO integration, and dedicated support.
- **Inference Endpoints**: Instantly deploy models with optimized, scalable infrastructure.
- **Pricing**: Flexible plans, starting at $0.60/hour for compute and $20/user/month for team features.

Join 50,000+ organizations—including **Meta, Amazon, Google, Intel, Microsoft, Grammarly, and NGO AI2**—trusting Hugging Face to accelerate their AI journey.

---

## Why Choose Hugging Face?

- **Open Source Leadership**: Building foundational AI tooling, in partnership with the world's researchers and practitioners.
- **Multi-Modality**: Work across text, images, audio, video, and 3D.
- **Rapid Innovation**: Access trending models, state-of-the-art research, and a vibrant community.
- **Enterprise Grade**: Security, compliance, and scalability tailored to organizations.

---

## Our Culture

- **Open & Inclusive**: We believe the future of AI should be collaborative and accessible to all.
- **Community-Driven**: Co-create and learn with contributors worldwide.
- **Impact-Oriented**: We champion ethical, safe, and effective machine learning practices.

---

## Careers at Hugging Face

Join a world-class team building the future of machine learning! We offer:

- A mission-driven, international environment
- Opportunities to work at the cutting edge of open-source AI
- Remote and flexible work arrangements
- Competitive compensation and benefits

[Explore Careers at Hugging Face →](https://huggingface.co/jobs)

---

## Let’s Build the Future Together

Whether you’re a developer, researcher, organization, or just curious, Hugging Face invites you to explore, build, and transform what’s possible with AI.

- [Sign Up](https://huggingface.co/join)
- [Explore Models & Datasets](https://huggingface.co/models)
- [Contact Us for Enterprise Solutions](https://huggingface.co/enterprise)

---

*Hugging Face — The platform where the machine learning community collaborates on models, datasets, and applications.*



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>